In [1]:
modello_per_analisi = 'Mercedes_Classe_A'
# Opzioni possibili: Mercedes_Classe_A, BMW_Serie_1, Audi_A3, Fiat_Panda

### 0. Import funzioni e librerie

In [2]:
import pandas as pd
import os
import sys
import requests
import importlib
from pathlib import Path


In [3]:
project_root = Path.cwd().parents[1]

# Percorsi delle cartelle contenenti i moduli
loader_dir = project_root / 'Source' / '2_Data_Preparation' / 'Utils'
utils_dir = project_root / 'Source' / '3_Data_Analysis' / 'Utils'
# Aggiungi al sys.path
sys.path.extend([str(loader_dir), str(utils_dir)])

# Import
from data_loader import load_csv_for_notebook
from Plots_maker import plot_is_conveniente_distribution, plot_price_prediction_distributions
from Function_price_analysis import modello_ml, predizione_prezzo, is_conveniente_class_report, indice_appetibilita, display_top_auto

In [4]:
# --- IMPORT CONFIGURAZIONE DELL'AUTO ---
#Configurazione path per import dinamico del modulo di configurazione 

config_path = project_root / 'Data'/ 'config'

sys.path.extend([str(config_path)])
print("\n - config_path:", config_path)
print(" - Contenuto della cartella:", os.listdir(config_path))

# --- Import dinamico del modulo di configurazione in base al modello ---
modulo_config = f"config_{modello_per_analisi}"  # esempio: config_Mercedes_Classe_A
try:
    config = importlib.import_module(modulo_config)
except ModuleNotFoundError:
    print(f"❌ Modulo di configurazione '{modulo_config}' non trovato.")
    sys.exit(1)
# --- Accesso ai parametri di configurazione ---
#allestimenti = config.allestimenti
allestimento_performance = config.allestimento_performance
allestimento_sport = config.allestimento_sport
allestimento_middle = config.allestimento_middle
allestimento_base = config.allestimento_base


 - config_path: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/config
 - Contenuto della cartella: ['config_Audi_A3.py', 'config_Opel_Corsa.py', '__pycache__', 'config_BMW_Serie_1.py', 'config_Mercedes_Classe_A.py']


### 1. Dati

In [5]:
#### CARICAMENTO DATI SINGOLI
data_dummy= load_csv_for_notebook(project_root,['Data', 'Processed_data', modello_per_analisi, f'data_dummy_{modello_per_analisi}.csv'])
data = load_csv_for_notebook(project_root,['Data', 'Processed_data', modello_per_analisi, f'data_{modello_per_analisi}.csv'])

X = data_dummy.drop(columns=['Prezzo', 'Distanza', 'Venditore'])
y = data_dummy['Prezzo']

✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/Mercedes_Classe_A/data_dummy_Mercedes_Classe_A.csv (3043 righe, 15 colonne)
✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/Mercedes_Classe_A/data_Mercedes_Classe_A.csv (3043 righe, 18 colonne)


### 2. Predizione prezzo, analisi della convenienza

In [6]:
modello_ml(X, y, (project_root / 'Data' / 'Modelli_ML' / modello_per_analisi))
data_ml, y_pred = predizione_prezzo(data_dummy, X, (project_root / 'Data' / 'Modelli_ML' / modello_per_analisi))

Valutazione delle performance sul test set
 - Mean Squared Error: 5146608.2
 - Root Mean Squared Error: 2268.61 euro
 - R-squared:  0.92


In [7]:
plot_is_conveniente_distribution(data_ml)


Distribuzione relativa della variabile 'is_conveniente':

 is_conveniente  Frequenza Relativa
              0            0.546172
              1            0.453828


In [8]:
plot_price_prediction_distributions(data_ml)

In [9]:
#is_conveniente_class_report(data_ml)

### 3. Indice di attendibilità

In [10]:
#Prezzo soglia: prezzo oltre il quale il punteggio di attendibilità diminuisce
prezzo_soglia=35000

#Valori per il calcolo indice di attendibilità
valore_idx_anni = 0.025
valore_idx_prezzo = 0.15
valore_idx_km= 0.15 
valore_idx_dist = 0.10 
valore_idx_allestimento= 0.25 
valore_idx_cv =0.025
valore_idx_cambio = 0.20 

In [11]:
df = indice_appetibilita(data, y_pred, prezzo_soglia, valore_idx_anni, valore_idx_prezzo, valore_idx_km, 
                           valore_idx_dist, valore_idx_allestimento, valore_idx_cv, valore_idx_cambio, 
                           allestimento_performance, allestimento_sport,allestimento_middle,allestimento_base)

In [12]:
prezzo_max = 32000
prezzo_min = 24000
km_max =40000
km_min = 0
dist_max = 150
dispaly = 20

display_top_auto(df, dispaly, prezzo_max, prezzo_min, km_max, km_min, dist_max, carburante=None)


Le migliori 20 auto secondo l'indice di appetibilità sono:


,Annuncio,Link,Prezzo,Anni,Immatricolazione,Chilometraggio,Cambio,Carburante,CV,Allestimento,...,Venditore,Distanza,Comune,CAP,Regione,Area,Indice_Appetibilità,prezzo_previsto,delta_prezzo,is_conveniente
41,Mercedes-Benz A 180 Premium AMG VARI COLORI E ...,https://www.autoscout24.it/annunci/mercedes-be...,29900,1,2024,15500,automatico,Benzina,150,AMG line,...,Rivenditore,77,arcole,37040,Veneto,Nord-est,0.81,31985.50,2085.50,1
261,Mercedes-Benz A 200 Premium auto,https://www.autoscout24.it/annunci/mercedes-be...,27800,1,2024,38000,automatico,Benzina,150,AMG line,...,Privato,25,bologna,40141,Emilia-Romagna,Nord-est,0.81,30246.79,2446.79,1
9,Mercedes-Benz A 180 d PREMIUM AMG AUTOMATIC,https://www.autoscout24.it/annunci/mercedes-be...,29900,3,2022,11500,automatico,Diesel,150,AMG line,...,Rivenditore,97,altavilla vicentina,36077,Veneto,Nord-est,0.80,32674.07,2774.07,1
10,Mercedes-Benz A 180 MHV PREMIUM AMG AUTOMATIC,https://www.autoscout24.it/annunci/mercedes-be...,30900,1,2024,23824,automatico,Ibrido,150,AMG line,...,Rivenditore,97,altavilla vicentina,36077,Veneto,Nord-est,0.80,31591.50,691.50,1
32,Mercedes-Benz A 180 MHV PREMIUM AMG AUTOMATIC,https://www.autoscout24.it/annunci/mercedes-be...,31700,1,2024,14728,automatico,Ibrido,150,AMG line,...,Rivenditore,97,altavilla vicentina,36077,Veneto,Nord-est,0.80,36589.00,4889.00,1
33,Mercedes-Benz A 180 MHV PREMIUM AMG AUTOMATIC,https://www.autoscout24.it/annunci/mercedes-be...,31900,1,2024,23818,automatico,Ibrido,150,AMG line,...,Rivenditore,97,altavilla vicentina,36077,Veneto,Nord-est,0.80,32151.50,251.50,1
40,Mercedes-Benz A 180 AMG PREMIUM LED NIGHT VARI...,https://www.autoscout24.it/annunci/mercedes-be...,28800,3,2022,23400,automatico,Benzina,150,AMG line,...,Rivenditore,77,arcole,37040,Veneto,Nord-est,0.80,29801.00,1001.00,1
42,Mercedes-Benz A 180 PREMIUM AMG PACK LUCI VARI...,https://www.autoscout24.it/annunci/mercedes-be...,30500,2,2023,19000,automatico,Benzina,150,AMG line,...,Rivenditore,77,arcole,37040,Veneto,Nord-est,0.80,31875.50,1375.50,1
48,Mercedes-Benz A 250 e Premium VARI COLORI E AL...,https://www.autoscout24.it/annunci/mercedes-be...,29900,2,2023,18300,automatico,Ibrido,150,AMG line,...,Rivenditore,77,arcole,37040,Veneto,Nord-est,0.80,32037.00,2137.00,1
86,Mercedes-Benz A 180 A 180 d Premium auto,https://www.autoscout24.it/annunci/mercedes-be...,28900,2,2023,29236,automatico,Diesel,150,AMG line,...,Rivenditore,65,parma,43126,Emilia-Romagna,Nord-est,0.80,31021.00,2121.00,1
